In [23]:
import tarfile

# Define file paths
model_files = ['knn_model.pkl', 'scaler.pkl', 'label_encoder.pkl', 'inference.py']  # Add all your model files here
tar_file_path = 'model_files.tar.gz'

# Create a tar.gz archive
with tarfile.open(tar_file_path, 'w:gz') as tar:
    for file in model_files:
        tar.add(file)

print("Model archive created successfully:", tar_file_path)


Model archive created successfully: model_files.tar.gz


In [25]:
import boto3
import os
from dotenv import load_dotenv

# Step 1: Load environment variables from .env
load_dotenv()

# Step 2: Get credentials from environment variables
aws_access_key = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
aws_region = os.getenv('AWS_REGION')

# Step 3: Initialize the S3 client
s3_client = boto3.client(
    's3',
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region
)

# Step 4: Define file and bucket details
local_file_path = 'model_files.tar.gz'          # Path to the zip file
bucket_name = 'music-recommender'             # Name of the S3 bucket
s3_file_key = 'models/model_files.tar.gz'       # Key (path) in the S3 bucket

# Step 5: Upload the file to S3
try:
    s3_client.upload_file(local_file_path, bucket_name, s3_file_key)
    print(f"File uploaded successfully to s3://{bucket_name}/{s3_file_key}")
except Exception as e:
    print(f"Error uploading file: {e}")


File uploaded successfully to s3://music-recommender/models/model_files.tar.gz


In [27]:
import boto3
import os
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# AWS region and role information
aws_region = os.getenv('AWS_REGION', 'us-east-1')
execution_role_arn = 'arn:aws:iam::024848456817:role/SageMakerExecutionRole'

# Initialize the SageMaker client
sagemaker_client = boto3.client('sagemaker', region_name=aws_region)

# Define model details
model_name = 'custom-music-recommender-model'
s3_model_path = 's3://music-recommender/models/model_files.tar.gz'
sagemaker_client.delete_model(ModelName=model_name)
# Specify script mode container for Python
response = sagemaker_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=execution_role_arn,
    PrimaryContainer={
       'Image': '683313688378.dkr.ecr.' + aws_region + '.amazonaws.com/sagemaker-scikit-learn:1.0-1-cpu-py3',
  # Update this if using another image
        'ModelDataUrl': s3_model_path,
        'Environment': {
            'SAGEMAKER_PROGRAM': 'inference.py',  # Name of your custom script
            'SAGEMAKER_SUBMIT_DIRECTORY': s3_model_path
        }
    }
)

print("Model created successfully:", response['ModelArn'])


Model created successfully: arn:aws:sagemaker:us-east-1:024848456817:model/custom-music-recommender-model


In [32]:
endpoint_config_name = 'music-recommender-endpoint-config'
sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
print("Creating endpoint configuration...")
response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'AllTraffic',
            'ModelName': model_name,
            'InstanceType': 'ml.m5.large',
            'InitialInstanceCount': 1
        }
    ]
)
print("Endpoint configuration created successfully:", response['EndpointConfigArn'])

Creating endpoint configuration...
Endpoint configuration created successfully: arn:aws:sagemaker:us-east-1:024848456817:endpoint-config/music-recommender-endpoint-config


In [36]:
endpoint_name = 'music-recommender-endpoint'
print("Creating endpoint...")
response = sagemaker_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)
print("Endpoint creation initiated:", response['EndpointArn'])


Creating endpoint...
Endpoint creation initiated: arn:aws:sagemaker:us-east-1:024848456817:endpoint/music-recommender-endpoint


In [37]:
#checking the status of the endpoint
import boto3
import json

# Initialize SageMaker client
sagemaker_client = boto3.client('sagemaker', region_name='us-east-1')

# Get endpoint details
response = sagemaker_client.describe_endpoint(EndpointName='music-recommender-endpoint')
print(json.dumps(response, indent=4, default=str))


{
    "EndpointName": "music-recommender-endpoint",
    "EndpointArn": "arn:aws:sagemaker:us-east-1:024848456817:endpoint/music-recommender-endpoint",
    "EndpointConfigName": "music-recommender-endpoint-config",
    "EndpointStatus": "Creating",
    "CreationTime": "2025-02-02 06:14:10.555000-05:00",
    "LastModifiedTime": "2025-02-02 06:14:10.986000-05:00",
    "ResponseMetadata": {
        "RequestId": "2decc515-4232-4704-8e3b-2afb31100fd8",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "x-amzn-requestid": "2decc515-4232-4704-8e3b-2afb31100fd8",
            "content-type": "application/x-amz-json-1.1",
            "content-length": "291",
            "date": "Sun, 02 Feb 2025 11:14:15 GMT"
        },
        "RetryAttempts": 0
    }
}


In [38]:
import time
def wait_for_endpoint(endpoint_name):
    print("Waiting for endpoint to become active...")
    while True:
        response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
        status = response['EndpointStatus']
        print(f"Endpoint status: {status}")

        if status == 'InService':
            print("Endpoint is ready!")
            break
        elif status == 'Failed':
            raise Exception("Endpoint deployment failed")

        time.sleep(30)

wait_for_endpoint(endpoint_name)


Waiting for endpoint to become active...
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: InService
Endpoint is ready!


In [ ]:
#test